# Подготовка входных данных
В спорте сложно подготовить универсального спортсмена даже в одной области: большинство бегунов специализируется на конкретной дистанции. Большинство пловцов показывают хорошие результаты только в одном стиле плавания.
Похожая ситуация и в других областях: чаще всего человек оказывается лучшим в чем-то одном (наука, исскуство).

Эту аналогию можно распространить и на машинное обучение: сложно построить универсальную модель, которая будет работать со всевозможными данными, пусть даже и из одной области.

Обойти эту проблему можно несколькими способами. Можно разделить исходную задачу на более мелкие (чтобы входные данные были более однородны) и обучить моедели для каждой задачи отдельно. Однако это сильно усложнит и удлинит процесс разработки.

Другой подход - обучать одну модель, но сделать данные более однородными, привести их к "стандартному" виду. Такой подход называется нормализацией входных данных.

Что можно считать стандартными значениями для переменной вещественной переменной?

1. Интервал $(-\infty, \infty)$ - слишком большой разброс, неудобно
2. Интервал $[0, \infty)$ - чуть лучше, но все равно проблемно
3. Интервал $[0, 1]$ -   переодически используется. Недостаток: многие соотношения используют "знак" выражения, при таком выборе нет симметрии между отрицательными и положительными значениями
4. Интервал $[-1,1]$ - используется. Недостаток: представим сеюе набор данных, в котором есть значение -1000, затем значения из промежутка $[-1,1]$ и последние - 1000. Если такой набор сжать в промежуток $[-1,1]$, то основные данные будут лежать в очень маленькой его части.


    




Основной подход:

1. Вычислим среднее арифметическое значение данных. Сохраним его в переменную mean. Вычтем среднее из всех значений, тогда среднее нового набора будет 0. То есть данные будут лежать симметрично относительно 0.
2. Посчитаем насколько начальные значения отличаются от среднего: вычтем из начальных данных среднее и каждое выражение возведем в квадрат:
$$\Delta x_i^2= (x_i-mean)^2$$
3. Найдем среднее таких квадратичных отклонений и извлечем из него корень:
$$\sigma = \sqrt{\frac{\Sigma_i \Delta x_i^2}{N}}$$

    Тогда если начальные данные были распределены нормально, то в промежуток $[-\sigma, \sigma]$ попадет ~68% всех значений.
    ![Распределение данных при нормальном распределении](https://raw.githubusercontent.com/mlforschool/lessons/master/6_Norm_and_Regular/Images/Standard_deviation_diagram.svg.png)
    (Изображение взято со страницы https://en.wikipedia.org/wiki/Standard_deviation#/media/File:Standard_deviation_diagram.svg)

    4. Теперь мы можем поделить входные данные на $\sigma$ и тогда ~68%  в промежутке [-1,1].
    5. На нормаллизованнных данных модель будет обучаться быстрее и давать более точные прогнозы.
    
    
    Такой алгоритм нужно применить к каждой компоненте входных данных: если каждый объект рписывается 3 значениями, то нужно посчитать три средних, три $\sigma$ и выполнить 3 нормализации
    
    
    
    

In [0]:
import numpy as np

def generate_data(size = 10):
    # случайные точки
    # числа из диапазона [0,1]
    X = np.random.rand(size, 2) 
    
    X[:,0] = X[:,0] *0.5 + 5
    X[:,1] = X[:,1] *120 - 13 
    print(X)
    
    # задаем функцию для ответов
    mask = 0.12 * X[:, 1] **2 + X[:, 0] * 10 - X[:, 1] * 7
    
    # добавляем шум
    mask = mask + np.random.rand(size) * 0.2
    
    
    # классы
    y = np.zeros(size, dtype=int)
    y[mask >= 0] = 1
    y[mask < 0] = 0
    
    
    
    return X, y

## Задание

Сгенерируйте данные 100 точек и приведите их стандартному виду с помощью описанного выше алгоритма

In [0]:
# начальные данный
X, y = None, None

# считаем средние
# не забудьте, что у команды mean есть параметр axis, который указывает ось по кторой считается среднее
mean = None

# вычитаем среднее
X1 = None

# возводим все разности в квадрат
dX = None

# находим среднее от разностей
# не забудьте, что у команды mean есть параметр axis, который указывает ось по кторой считается среднее
var = None


# извлекаем корень:
sigmas = None

# делим значения на сигмы и получаем нормализованные данные
res = None


In [0]:
# существуют уже готовые пакеты для нормализации данных
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
res2 = sc.fit_transform(X)

# сравните свой res и res2, их разность должна получится 0
if res and res2:
    print(res - res2)


Используем данные для обучения логистической регрессии

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [0]:
# не нормализованные данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

lr = LogisticRegression().fit(X_train, y_train)

print(lr.score(X_test, y_test))

In [0]:
# нормализованные данные
X_train2, X_test2, y_train2, y_test2 = train_test_split(res, y, test_size=0.33, random_state=42)
lr2 = LogisticRegression().fit(X_train2, y_train2)

lr2.score(X_test2, y_test2)

Различие минимально, но оно есть.

Чем более сложные и нелинейные модели мы будем использовать, тем чуствительнее они будут к нормализации данных


# Задание 2
Выполните обратное преобразование данных.
Используя mean и sigma получите из нормализованных  данных исходные

In [0]:
# нормализованные данные
res, mean, sigmas

# ваш код


X_new = None


# сравнение, максимальный модуль разности
# должен быть 0
if X and X_new:
    print(np.max(np.abs(X - X_new))
